### Extracción de datos desde API de Spotify

Crear un archivo config.py en el que se guarden las siguientes credenciales: CLIENT_ID y CLIENT_SECRET.

In [ ]:
import requests
import base64
from config import CLIENT_ID, CLIENT_SECRET

client_id = CLIENT_ID
client_secret = CLIENT_SECRET

# Codificar las credenciales en base64
credentials = base64.b64encode(f"{client_id}:{client_secret}".encode()).decode("utf-8")

# Hacer la solicitud para obtener el token
url = "https://accounts.spotify.com/api/token"
headers = {
    "Authorization": f"Basic {credentials}"
}
data = {
    "grant_type": "client_credentials"
}

response = requests.post(url, headers=headers, data=data)
token = response.json().get("access_token")

print(f"Token de acceso: {token}")

Token de acceso: BQBWWxOlMwsTR2OstVJ5YN4eoCvP38HdAaXOcmkX2a57wI3aMUfcIH7LM_ixgLadHnMlz5_VFXJOdSwUe04n1daYu49rpOIfckY_ZukgnRR0Fk1e3Do


Leer de la base de datos SQL Server la data extraída en la primera parte.

In [ ]:
import pyodbc
import pandas as pd
import requests
from config import SERVER, DATABASE

conn_str = (
    r'DRIVER={SQL Server};'
    rf'SERVER={SERVER};' 
    rf'DATABASE={DATABASE};'
    r'Trusted_Connection=yes;'
)
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

# Leer los datos de la tabla Track
cursor.execute("SELECT Nombre_Artista, Nombre_Album, Nombre_Track, ID, Duration_ms, Explicit, Artists, Track_Number FROM Track")
tracks = cursor.fetchall()

In [3]:
tracks

[('Shakira', 'Las Mujeres Ya No Lloran', 'Puntería', '2dYXrWkNO40NXy2Q3IYLRc', 181666, False, 'Shakira, Cardi B', 1),
 ('Shakira', 'Las Mujeres Ya No Lloran', 'La Fuerte', '09mHLXMh3rmruStxevfh3U', 164833, False, 'Shakira, Bizarrap', 2),
 ('Shakira', 'Las Mujeres Ya No Lloran', 'Tiempo Sin Verte', '6c8jkAgbVk44B8SwRi8vQs', 196233, False, 'Shakira', 3),
 ('Shakira', 'Las Mujeres Ya No Lloran', 'Cohete', '5JKk6Gh5CLlwpbPz48eWtB', 172503, False, 'Shakira, Rauw Alejandro', 4),
 ('Shakira', 'Las Mujeres Ya No Lloran', '(Entre Paréntesis)', '68viiGZoRbLCjsy5xD4VrU', 168896, False, 'Shakira, Grupo Frontera', 5),
 ('Shakira', 'Las Mujeres Ya No Lloran', 'Cómo Dónde y Cuándo', '40Ccxt7rKCXiViGQKPPACR', 179000, False, 'Shakira', 6),
 ('Shakira', 'Las Mujeres Ya No Lloran', 'Nassau', '2D48pEtMZRYcByAhjvVLM3', 156466, False, 'Shakira', 7),
 ('Shakira', 'Las Mujeres Ya No Lloran', 'Última', '3FEIMzqeq5EXqaMi3mHP6L', 178666, False, 'Shakira', 8),
 ('Shakira', 'Las Mujeres Ya No Lloran', 'Te Felicito

In [4]:
def get_audio_features(track_id, token):
    url = f"https://api.spotify.com/v1/audio-features/{track_id}"
    headers = {
        "Authorization": f"Bearer {token}"
    }

    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        return {
            "acousticness": data.get("acousticness"),
            "danceability": data.get("danceability"),
            "energy": data.get("energy"),
            "instrumentalness": data.get("instrumentalness"),
            "liveness": data.get("liveness"),
            "loudness": data.get("loudness"),
            "speechiness": data.get("speechiness"),
            "tempo": data.get("tempo"),
            "valence": data.get("valence"),
            "analysis_url": data.get("analysis_url"),
            "key": data.get("key"),
            "mode": data.get("mode"),
            "time_signature": data.get("time_signature")
        }
    else:
        return {}

def get_track_info(track_id, token):
    url = f"https://api.spotify.com/v1/tracks/{track_id}"
    headers = {
        "Authorization": f"Bearer {token}"
    }

    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        return {
            "popularity": data.get("popularity"),
            "name": data.get("name"),
            "album": data.get("album", {}).get("name"),
            "artists": [artist.get("name") for artist in data.get("artists", [])]
        }
    else:
        return {}


In [5]:
##### Asegurarse de haber corrido la primera celda
##### Para tener token de Spotify

for track in tracks:
    features = get_audio_features(track.ID, token)
    track_info = get_track_info(track.ID, token)
    
    cursor.execute("""
        INSERT INTO Track_Features_2 (Nombre_Artista, Nombre_Album, Nombre_Track, ID, Duration_ms, Explicit, Artists, Track_Number,
                                    acousticness, danceability, energy, instrumentalness, liveness, loudness, speechiness, tempo, valence, analysis_url, [key], mode, time_signature, popularity)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, (
        track.Nombre_Artista, track.Nombre_Album, track.Nombre_Track, track.ID, track.Duration_ms, track.Explicit, track.Artists, track.Track_Number,
        features.get("acousticness"), features.get("danceability"), features.get("energy"), features.get("instrumentalness"),
        features.get("liveness"), features.get("loudness"), features.get("speechiness"), features.get("tempo"),
        features.get("valence"), features.get("analysis_url"), features.get("key"), features.get("mode"), features.get("time_signature"),track_info.get("popularity")
    ))

conn.commit()
cursor.close()
conn.close()
